### 순서
1. 문서의 내용을 읽는다.
2. 문서를 쪼갠다.
    - 토큰수 초과로 답변을 생성하지 못할 수 있고
    - 문서가 길면(입력이 길면) 답변 생성이 오래걸림
3. 임베딩 -> 벡터 데이터베이스에 저장
4. 질문이 있을 때, 벡터 데이터베이스에 유사도 검색
5. 유사도 검색으로 가져온 문서를 LLM에 질문과 함께 전달

In [4]:
# %pip install -qU  docx2txt langchain_community

In [5]:
# %pip install -qU langchain-text-splitters

In [ ]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import Docx2txtLoader

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1500,
    chunk_overlap=200,
)

loader = Docx2txtLoader("./tax.docx")
document = loader.load() # chunking 없이 한번에 불러옴
document_list = loader.load_and_split(text_splitter=text_splitter) # chunking 하여 불러옴

In [7]:
len(document) # chunking 없이 한번에 불러옴


1

In [8]:
len(document_list) # chunking 하여 불러옴

194

In [9]:
from dotenv import load_dotenv
from langchain_google_genai import GoogleGenerativeAIEmbeddings

load_dotenv()

embedding = GoogleGenerativeAIEmbeddings(model="models/gemini-embedding-001")

In [10]:
# %pip install -qU langchain-chroma

In [11]:
# %pip install langchain-pinecone

In [ ]:
import os

from pinecone import Pinecone
from langchain_pinecone import PineconeVectorStore

# INDEX_NAME = "tax-index"
PINECONE_API_KEY = os.environ.get("PINECONE_API_KEY")
pc = Pinecone(api_key=PINECONE_API_KEY)

index = pc.Index(INDEX_NAME)

In [ ]:
database = PineconeVectorStore(index=index, embedding=embedding)
database.add_documents(document_list)

In [20]:
query = "연봉 5천만원인 직장인의 소득세는 얼마인가요?"
retrieved_docs = database.similarity_search(query, k=3)

In [21]:
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash")

In [ ]:
# prompt = f"""[identity]
# - 당신은 최그의 한국 소득세 전문가입니다
# - [context]를 참고해서 사용자의 질문에 답변해주세요

# [context]
# {retrieved_docs}

# question: {query}
# """

In [ ]:
# ai_message = llm.invoke(prompt)

In [ ]:
# ai_message.content

In [22]:
import os

load_dotenv()

LANGSMITH_API_KEY = os.getenv("LANGSMITH_API_KEY")

In [23]:
from langsmith import Client

client = Client(api_key=LANGSMITH_API_KEY)

prompt = client.pull_prompt("rlm/rag-prompt")

In [24]:
prompt

ChatPromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, metadata={'lc_hub_owner': 'rlm', 'lc_hub_repo': 'rag-prompt', 'lc_hub_commit_hash': '50442af133e61576e74536c6556cefe1fac147cad032f4377b60c436e6cdcb6e'}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: {question} \nContext: {context} \nAnswer:"), additional_kwargs={})])

# 여기서부터 중단.

(문제1-해결)hub이 deprecate 됐고 langsmith를 대체로 활용해야하는데 문서가 복잡해 일단 hub를 왜 쓰는지에 대한 다음 강의 수강후 다시 돌아와 문서 공부 필요.  


(문제2)RetrievalQA 가 deprecate 됐는데 이게 뭘 하는 건지, 그리고 뭔지 모르니 뭘로 대체해야 할지도 모르는 상태.

관련 검색:
- 검색어 RetrievalQA
- 검색어 RetrievalQA.from_chain_type()

### 1. 레거시 retrievalQA

In [ ]:
# from langchain.chains import RetrievalQA

# qa_chain = RetrievalQA.from_chain_type(
#     llm, 
#     retriever=database.as_retriever(),
#     chain_type_kwargs={"prompt": prompt}
# )

In [ ]:
# ai_message = qa_chain({"query": query})

In [ ]:
# ai_message

### 2. langchain 공식문서 검색에서 찾은 agent 방식(불가)

In [ ]:
# from langchain.agents import create_agent

# tools = [retrieve_context]
# # If desired, specify custom instructions
# prompt = (
#     "You have access to a tool that retrieves context from a blog post. "
#     "Use the tool to help answer user queries."
# )

# agent = create_agent(llm, database, system_prompt=prompt)

### 3. gpt 질의 통해 얻은 LCEL 방식

In [27]:
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

# 1. Retriever 설정 (검색기)
retriever = database.as_retriever()

# 2. 문서들을 하나로 합치는 헬퍼 함수
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

# 3. LCEL 체인 구성 (이것이 현대적인 RetrievalQA입니다)
# 흐름: {context: 검색결과, question : 질문} -> 프롬프트 -> LLM -> 문자열 변환
rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

# 4. 실행
# invoke에 문자열(query)만 넣으면 RunnablePassthrough가 받아서 처리합니다.
ai_message = rag_chain.invoke(query)

print(ai_message)

제공된 소득세법 조항에는 연봉 5천만원 직장인의 소득세를 직접 명시하거나 계산한 내용은 없습니다. 다만, 근로소득공제, 세율, 근로소득세액공제 등 소득세 계산에 필요한 규정들이 설명되어 있습니다. 따라서 해당 정보만으로는 정확한 소득세를 알 수 없습니다.
